In [1]:
from FastFitClassifier import FastFitClassifier
import pandas as pd
from tqdm.auto import tqdm

tqdm.pandas()

In [2]:
DEV = True
model_path = "./models/relevance/avsolatorio/GIST-small-Embedding-v0"



if not DEV:
    model_path = './models/relevance/avsolatorio/GIST-Embedding-v0'

In [3]:
def import_labelled_data(path="data/level-0.5/data.json"):
    data = pd.read_json(path, encoding="latin-1")
    return data



print("Loading data...")

test_dataset = import_labelled_data(path="../../../data/level-0.5/irrelevant.json", )

print("Data loaded.")

if DEV:
    test_dataset = test_dataset.sample(5000)

test_dataset = test_dataset.sample(frac=1).reset_index(drop=True)


test_dataset.info()

Loading data...
Data loaded.
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5000 entries, 0 to 4999
Data columns (total 4 columns):
 #   Column        Non-Null Count  Dtype 
---  ------        --------------  ----- 
 0   url           5000 non-null   object
 1   text          5000 non-null   object
 2   relevance     5000 non-null   object
 3   multiclasses  5000 non-null   object
dtypes: object(4)
memory usage: 156.4+ KB


In [4]:
test_dataset.head()

,url,text,relevance,multiclasses
0,http://repositorio.unap.edu.pe/bitstream/handl...,UNIVERSIDAD NACIONAL DEL ALTIPLANO - PUNO FACU...,irrelevant,[]
1,https://www.wycokck.org/files/assets/public/v/...,"ARPA Facilities Review February 8, 2022 1 $91....",irrelevant,[]
2,https://www.chelsealogistics.ph/wp-content/upl...,The following document has been received: Rece...,irrelevant,[]
3,https://www.letran.edu.ph/Documents/AY2023-202...,www.letran.edu.ph/Home/Resources Tuesday - Jul...,irrelevant,[]
4,https://deepdivestocks.com/wp-content/uploads/...,$SPY on 6/17/2022 VoEx Graph 5.0 2.5 x E 0.0 o...,irrelevant,[]


In [5]:
import torch
torch.cuda.empty_cache()

In [6]:
classifier = FastFitClassifier(
    model_path=model_path,
    text_overlap_proportion=0.2,
    device='cuda'
)

Loading model from ./models/relevance/avsolatorio/GIST-small-Embedding-v0 ...
Model loaded.
Tokenizer loaded....


The model 'FastFit' is not supported for text-classification. Supported models are ['AlbertForSequenceClassification', 'BartForSequenceClassification', 'BertForSequenceClassification', 'BigBirdForSequenceClassification', 'BigBirdPegasusForSequenceClassification', 'BioGptForSequenceClassification', 'BloomForSequenceClassification', 'CamembertForSequenceClassification', 'CanineForSequenceClassification', 'LlamaForSequenceClassification', 'ConvBertForSequenceClassification', 'CTRLForSequenceClassification', 'Data2VecTextForSequenceClassification', 'DebertaForSequenceClassification', 'DebertaV2ForSequenceClassification', 'DistilBertForSequenceClassification', 'ElectraForSequenceClassification', 'ErnieForSequenceClassification', 'ErnieMForSequenceClassification', 'EsmForSequenceClassification', 'FalconForSequenceClassification', 'FlaubertForSequenceClassification', 'FNetForSequenceClassification', 'FunnelForSequenceClassification', 'GemmaForSequenceClassification', 'Gemma2ForSequenceClassif


Classifier pipeline built.


#### Profiling Section

In [7]:
!pip install line_profiler
%load_ext line_profiler

Looking in indexes: https://pypi.org/simple, https://pypi.ngc.nvidia.com

[notice] A new release of pip is available: 24.1.2 -> 24.2
[notice] To update, run: pip install --upgrade pip


In [8]:
from transformers.pipelines.pt_utils import KeyDataset
from datasets import Dataset
print("Chunking data...", end="\r")
from chunking import chunk_dataset_and_explode

data = test_dataset.sample(100)

chunked_data = chunk_dataset_and_explode(
    data, max_len=512, overlap=int(512 * 0.2)
)

# get dict of chunk_ids to counts of that id so we can track when we've finished all chunks for a file
chunk_id_counts = chunked_data["chunk_id"].value_counts().to_dict()

print("Data chunked.")

chunked_dataset = Dataset.from_pandas(chunked_data)

print("Predicting...", end="\r")

%lprun -u 1e-3 -f FastFitClassifier.predict_chunks classifier.predict_chunks(KeyDataset(chunked_dataset,'text'),chunk_id_counts, chunked_data['chunk_id'])

  0%|          | 0/100 [00:00<?, ?it/s]

Data chunked.


Files	:   0%|          | 0/100 [00:00<?, ?it/s]

Chunks	:   0%|          | 0/28580 [00:00<?, ?it/s]

Calculated predictions.


Timer unit: 0.001 s

Total time: 51.2633 s
File: /GreyLitDocker/GreyLiteratureClassifier/src/FastFit/level-1/FastFitClassifier.py
Function: predict_chunks at line 50

Line #      Hits         Time  Per Hit   % Time  Line Contents
    50                                               def predict_chunks(self, chunked_text, chunk_id_counts,chunk_ids):
    51         1          0.0      0.0      0.0          predictions = []
    52         1          0.0      0.0      0.0          scores = []
    53                                           
    54         1          0.0      0.0      0.0          i = 0
    55                                           
    56         1          0.1      0.1      0.0          print("Predicting from chunks...", end="\r")
    57                                           
    58         2         32.1     16.0      0.1          with tqdm(total=len(chunk_id_counts),desc='Files\t') as files_pbar:
    59         2         41.7     20.8      0.1              with t

##### End Profiling

In [9]:

#! on fastfit/modeling.py, line 838, inserting **kwargs
#! it's easier than getting the tokenizer to stop outputting
#! extra info like token_type_ids

predicted_data = classifier.evaluate(
    test_dataset,
    metrics=[
        "accuracy",
        "precision",
        "classification-report",
        "specificity",
        "confusion-matrix-mpl",
    ],
    aggregate="majority",
)

  0%|          | 0/5000 [00:00<?, ?it/s]

Data chunked.


Files	:   0%|          | 0/5000 [00:00<?, ?it/s]

Chunks	:   0%|          | 0/1516037 [00:00<?, ?it/s]

In [ ]:
potential = predicted_data[predicted_data["predictions"] == "relevant"]
potential = potential.sort_values(by="score-lv1", ascending=False)


In [ ]:
potential = potential[potential['text'].apply(lambda x: x!='No summary for this intervention.')]
potential = potential[potential['score-lv1'].apply(lambda x: x>=0.95)].reset_index(drop=True)

In [ ]:
# get max min range of score
max_score = potential['score-lv1'].max()
min_score = potential['score-lv1'].min()
print(f"Max score: {max_score}")
print(f"Min score: {min_score}")



Max score: 1.0
Min score: 0.9506133066283332


In [ ]:

if DEV:
    from os import makedirs, path
    if not path.exists("../../../data/level-1.5/potential/dev"):
        makedirs("../../../data/level-1.5/potential/dev")
    potential.to_json("../../../data/level-1.5/potential/dev/data.json", orient="records", indent=4)
else:
    potential.to_json("../../../data/level-1.5/potential/data.json", orient="records", indent=4)